# 1 - Comparing and reinforcing learning algorithms

compare the performance and characteristics of two key reinforcement learning algorithms—Q-learning and policy gradients. You will then perform an analysis to reinforce your understanding of how these algorithms work, when they are most effective, and how they handle learning tasks differently. 

## 1.1

Implement Q-learning
The first part of the activity focuses on Q-learning, a value-based reinforcement learning algorithm

In [1]:
import numpy as np

# Define the grid size and actions
grid_size = 5
n_actions = 4  # Actions: up, down, left, right

# Initialize the Q-table with zeros
Q_table = np.zeros((grid_size * grid_size, n_actions))

Step 2: Define the hyperparameters
Set the hyperparameters for Q-learning:

Learning rate (αα)

Discount factor (γγ)

Exploration rate (ϵϵ)

In [2]:
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor for future rewards
epsilon = 0.1  # Exploration rate for epsilon-greedy policy

Step 3: Define the reward structure
Create a reward matrix based on the environment's feedback:

+10 for reaching the goal

–10 for falling into the pit

–1 for other states

In [3]:
# Reward matrix for the grid environment
rewards = np.full((grid_size * grid_size,), -1)  # -1 for every state
rewards[24] = 10  # Goal state
rewards[12] = -10  # Pitfall state

In [4]:
def epsilon_greedy_action(Q_table, state, epsilon):
    if np.random.uniform(0, 1) < epsilon:
        return np.random.randint(0, n_actions)  # Explore: random action
    else:
        return np.argmax(Q_table[state])  # Exploit: action with highest Q-value

Update the Q-values
Use the Bellman equation to update the Q-values based on the current state, the selected action, and the reward obtained.

In [5]:
for episode in range(1000):
    state = np.random.randint(0, grid_size * grid_size)  # Start in a random state
    done = False
    while not done:
        action = epsilon_greedy_action(Q_table, state, epsilon)
        next_state = np.random.randint(0, grid_size * grid_size)  # Simulated next state
        reward = rewards[next_state]

        # Update Q-value using Bellman equation
        Q_table[state, action] = Q_table[state, action] + alpha * (reward + gamma * np.max(Q_table[next_state]) - Q_table[state, action])

        state = next_state
        if next_state == 24 or next_state == 12:
            done = True

# 2 - Implement policy gradients
The second part of the activity involves implementing policy gradients, a policy-based method in which the agent learns a policy (mapping from states to actions) by optimizing a neural network.

Step-by-step guide:
Step 1: Build the policy network
Define a neural network that takes the current state as input and then outputs a probability distribution over possible actions.

In [6]:
import tensorflow as tf

# Define the policy network
n_states = grid_size * grid_size  # Number of states in the grid
n_actions = 4  # Up, down, left, right

model = tf.keras.Sequential([
    tf.keras.layers.Dense(24, activation='relu', input_shape=(n_states,)),
    tf.keras.layers.Dense(n_actions, activation='softmax')  # Output action probabilities
])

# Optimizer for policy network updates
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

2025-06-03 11:09:43.524895: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 11:09:43.708090: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-03 11:09:43.838019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748948983.965704   30588 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748948983.999670   30588 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748948984.243724   30588 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Step 2: Select an action
For each state, the agent selects an action based on the probabilities output by the policy network.

In [7]:
def get_action(state):
    state_input = tf.one_hot(state, n_states)  # One-hot encoding for state
    action_probs = model(state_input[np.newaxis, :])
    return np.random.choice(n_actions, p=action_probs.numpy()[0])

In [8]:
# Simulation loop
states = []
actions = []
episode_rewards = []  

for episode in range(1000):
    state = np.random.randint(0, n_states)  # Start in a random state
    done = False
    while not done:
        action = get_action(state)  # Use the provided function
        next_state = np.random.randint(0, n_states)  # Simulated next state
        reward = rewards[next_state]  

        # Store the state-action-reward trajectory
        states.append(state)
        actions.append(action)
        episode_rewards.append(reward)  

        state = next_state
        if next_state in {24, 12}:  
            done = True

Step 4: Compute cumulative rewards
To reinforce actions that lead to long-term success, calculate the cumulative rewards for each action taken during an episode.

In [9]:
def compute_cumulative_rewards(rewards, gamma=0.99):
    cumulative_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(len(rewards))):
        running_add = running_add * gamma + rewards[t]
        cumulative_rewards[t] = running_add
    return cumulative_rewards

Step 5: Update the policy
Update the policy network using the REINFORCE algorithm based on the actions and cumulative rewards.

In [10]:
def update_policy(states, actions, rewards):
    cumulative_rewards = compute_cumulative_rewards(rewards)

    with tf.GradientTape() as tape:
        state_inputs = tf.one_hot(states, n_states)  # Convert states to one-hot encoding
        action_probs = model(state_inputs)
        action_masks = tf.one_hot(actions, n_actions)  # Mask for selected actions
        log_probs = tf.reduce_sum(action_masks * tf.math.log(action_probs), axis=1)

        # Policy loss is the negative log-probability of the action times the cumulative reward
        loss = -tf.reduce_mean(log_probs * cumulative_rewards)

    # Apply gradients to update the policy network
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

Step 4: Compute cumulative rewards
To reinforce actions that lead to long-term success, calculate the cumulative rewards for each action taken during an episode.

In [11]:
def compute_cumulative_rewards(rewards, gamma=0.99):
    cumulative_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(len(rewards))):
        running_add = running_add * gamma + rewards[t]
        cumulative_rewards[t] = running_add
    return cumulative_rewards

Step 5: Update the policy
Update the policy network using the REINFORCE algorithm based on the actions and cumulative rewards.

In [12]:
def update_policy(states, actions, rewards):
    cumulative_rewards = compute_cumulative_rewards(rewards)

    with tf.GradientTape() as tape:
        state_inputs = tf.one_hot(states, n_states)  # Convert states to one-hot encoding
        action_probs = model(state_inputs)
        action_masks = tf.one_hot(actions, n_actions)  # Mask for selected actions
        log_probs = tf.reduce_sum(action_masks * tf.math.log(action_probs), axis=1)

        # Policy loss is the negative log-probability of the action times the cumulative reward
        loss = -tf.reduce_mean(log_probs * cumulative_rewards)

    # Apply gradients to update the policy network
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [13]:
import matplotlib.pyplot as plt

# Example code to visualize rewards over episodes
plt.plot(rewards_q_learning, label='Q-Learning')
plt.plot(rewards_policy_gradients, label='Policy Gradients')
plt.xlabel('Episodes')
plt.ylabel('Cumulative Rewards')
plt.legend()
plt.show()

NameError: name 'rewards_q_learning' is not defined

Comparing Q-learning and policy gradients
In reinforcement learning, Q-learning and policy gradients are two popular approaches for training agents to make optimal decisions. While both methods aim to maximize cumulative rewards, they differ in their underlying mechanisms and are suited for different types of problems. Compare the methods below.

Speed of convergence
Q-learning tended to converge faster in this small grid environment. This is because Q-learning works well in environments with a discrete action space and fewer states, allowing the agent to build a reliable Q-table quickly.

Policy gradients required more episodes to stabilize because the agent learned the policy directly through gradient updates. However, policy gradients are more flexible in environments with continuous action spaces.

Reward maximization
Both algorithms eventually reached the goal consistently after enough episodes. However, Q-learning was more consistent in terms of reward maximization early on due to its more structured exploration.

Policy gradients started slowly but eventually caught up and produced comparable results.

Exploration vs. exploitation
Q-learning relies heavily on exploration through the epsilon-greedy policy. The agent systematically explored different paths, but it risked getting stuck in suboptimal actions when epsilon was too high.

Policy gradients did not explicitly balance exploration and exploitation; instead, they optimized the policy based on cumulative rewards, which naturally led to better action selection as the policy improved.

Suitability for different problems
Q-learning is more suited to environments with a small number of discrete actions and states, such as grid-based games or simple navigation tasks.

Policy gradients are better suited for environments with a continuous action space or more complex scenarios where approximating a value function (such as Q-values) becomes difficult.

Conclusion
In this walkthrough, you implemented and compared Q-learning and policy gradient algorithms in a simple grid environment. Both approaches demonstrated their strengths: Q-learning’s faster convergence and policy gradients' flexibility. While Q-learning is easier to implement in smaller, discrete environments, policy gradients offer a more scalable solution for larger, continuous action spaces.

By comparing both algorithms, you now have a deeper understanding of when to use each approach based on the complexity of the problem and the type of action space.